# Table 1

In [62]:
import pandas as pd
from pathlib import Path

# 1. Setup paths
file_path = Path("../Raw Data/560101.xlsx")
output_file = "../Datasets/df_total_loans.csv"

# 2. Load the file
df_full = pd.read_excel(file_path, sheet_name="Data1")

# 3. Define the Series IDs for Table 1 (Seasonally Adjusted Only)
# Mapping: (Segment, Buyer Type) -> {Count ID, Value ID}
mapping = [
    {"segment": "Total",          "buyer_type": "Total",               "count": "A130268892J", "value": "A130268891F"},
    {"segment": "Owner-occupier", "buyer_type": "Total",               "count": "A130268886L", "value": "A130268885K"},
    {"segment": "Investor",       "buyer_type": "Total",               "count": "A130268495C", "value": "A130268494A"},
    {"segment": "Owner-occupier", "buyer_type": "First Home Buyer",     "count": "A130268484W", "value": "A130268483V"},
    {"segment": "Owner-occupier", "buyer_type": "Non-First Home Buyer", "count": "A130268024J", "value": "A130268023F"},
]

# 4. Find the row index where Series IDs are located
# Searching for one of the known IDs to find the header row
id_row_index = df_full[df_full.apply(lambda row: row.astype(str).str.contains('A130268892J').any(), axis=1)].index[0]

# 5. Prepare data starting from row after the IDs
df_data = df_full.iloc[id_row_index + 1:].copy()
df_data.columns = df_full.iloc[id_row_index]
df_data = df_data.rename(columns={df_data.columns[0]: 'date'})

# 6. Tidy up the data
final_rows = []
for item in mapping:
    try:
        subset = df_data[['date', item['count'], item['value']]].copy()
        subset.columns = ['date', 'loan_count', 'loan_value_million']
        subset['segment'] = item['segment']
        subset['buyer_type'] = item['buyer_type']
        
        # --- REORDERING COLUMNS TO YOUR EXACT SPECIFICATION ---
        subset = subset[['date', 'segment', 'buyer_type', 'loan_count', 'loan_value_million']]
        
        final_rows.append(subset)
    except KeyError:
        print(f"Warning: Could not find IDs for {item['segment']} - {item['buyer_type']}")
        continue

# 7. Final Clean and Sort
df_final = pd.concat(final_rows, ignore_index=True)
df_final['date'] = pd.to_datetime(df_final['date'])

# Convert data to numeric
df_final['loan_count'] = pd.to_numeric(df_final['loan_count'], errors='coerce')
df_final['loan_value_million'] = pd.to_numeric(df_final['loan_value_million'], errors='coerce')

# Drop empty rows
df_final = df_final.dropna(subset=['loan_count', 'loan_value_million'], how='all')

# Sort
df_final = df_final.sort_values(['date', 'segment', 'buyer_type']).reset_index(drop=True)

# 8. Save to CSV
df_final.to_csv(output_file, index=False)

print(f"Extraction successful! Saved Table 1 as {output_file}")
print(df_final.head())

Extraction successful! Saved Table 1 as ../Datasets/df_total_loans.csv
        date         segment            buyer_type  loan_count  \
0 2002-09-01        Investor                 Total         NaN   
1 2002-09-01  Owner-occupier      First Home Buyer     24529.0   
2 2002-09-01  Owner-occupier  Non-First Home Buyer     64722.0   
3 2002-09-01  Owner-occupier                 Total     89705.0   
4 2002-09-01           Total                 Total         NaN   

   loan_value_million  
0             11009.5  
1              4121.6  
2             12468.5  
3             16698.5  
4             27708.0  


In [53]:
df_final.head()

,date,segment,buyer_type,loan_count,loan_value_million
0,2002-09-01,Investor,Total,NaN,11009.5
1,2002-09-01,Owner-occupier,First Home Buyer,24529.0,4121.6
2,2002-09-01,Owner-occupier,Non-First Home Buyer,64722.0,12468.5
3,2002-09-01,Owner-occupier,Total,89705.0,16698.5
4,2002-09-01,Total,Total,NaN,27708.0


In [54]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                465 non-null    datetime64[ns]
 1   segment             465 non-null    object        
 2   buyer_type          465 non-null    object        
 3   loan_count          329 non-null    float64       
 4   loan_value_million  465 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 18.3+ KB


# Table 2

In [63]:
import pandas as pd
from pathlib import Path

# 1. Setup paths
file_path = Path("../Raw Data/560103.xlsx")
output_file = "../Datasets/df_owner_occupier_purpose.csv"

# 2. Load the file
df_full = pd.read_excel(file_path, sheet_name="Data1")

# 3. Define the Series IDs (Seasonally Adjusted)
mapping = {
    "Total":                             {"count": "A130268886L", "value": "A130268885K"},
    "Construction of dwellings":         {"count": "A130268249W", "value": "A130268248V"},
    "Purchase of newly erected dwellings": {"count": "A130268251J", "value": "A130268250F"},
    "Purchase of existing dwellings":    {"count": "A130268036T", "value": "A130268035R"},
    "Purchase of residential land":      {"count": "A130267882K", "value": "A130267881J"},
    "Alterations, additions and repairs":{"count": "A130268189F", "value": "A130268188C"},
    "External refinancing":             {"count": "A130268811V", "value": "A130268810T"},
    "Internal refinancing":             {"count": "A130268247T", "value": "A130268246R"}
}

# 4. Find the row index where Series IDs are located
# This searches the whole sheet for one of our IDs to find the header row automatically
id_row_index = df_full[df_full.apply(lambda row: row.astype(str).str.contains('A130268886L').any(), axis=1)].index[0]

# 5. Prepare data starting from row after the IDs
df_data = df_full.iloc[id_row_index + 1:].copy()
df_data.columns = df_full.iloc[id_row_index]
df_data = df_data.rename(columns={df_data.columns[0]: 'date'})

# 6. Tidy up the data
final_rows = []
for purpose, ids in mapping.items():
    try:
        subset = df_data[['date', ids['count'], ids['value']]].copy()
        subset.columns = ['date', 'loan_count', 'loan_value_million']
        subset['loan_purpose'] = purpose
        subset['segment'] = "Owner-occupier"
        
        # --- REORDERING COLUMNS TO YOUR EXACT SPECIFICATION ---
        subset = subset[['date', 'segment', 'loan_purpose', 'loan_count', 'loan_value_million']]
        
        final_rows.append(subset)
    except KeyError:
        continue

# 7. Final Clean and Sort
df_final = pd.concat(final_rows, ignore_index=True)
df_final['date'] = pd.to_datetime(df_final['date'])

# Convert data to numeric types
df_final['loan_count'] = pd.to_numeric(df_final['loan_count'], errors='coerce')
df_final['loan_value_million'] = pd.to_numeric(df_final['loan_value_million'], errors='coerce')

# Remove empty historical rows (where data didn't exist yet)
df_final = df_final.dropna(subset=['loan_count', 'loan_value_million'], how='all')

# Sort chronologically
df_final = df_final.sort_values(['date', 'loan_purpose']).reset_index(drop=True)

# 8. Save to CSV
df_final.to_csv(output_file, index=False)

print(f"Extraction successful! Saved as {output_file}")
print(df_final.head())

Extraction successful! Saved as ../Datasets/df_owner_occupier_purpose.csv
        date         segment                         loan_purpose  loan_count  \
0 2002-09-01  Owner-occupier   Alterations, additions and repairs         NaN   
1 2002-09-01  Owner-occupier            Construction of dwellings      9777.0   
2 2002-09-01  Owner-occupier                 External refinancing     32454.0   
3 2002-09-01  Owner-occupier       Purchase of existing dwellings     77142.0   
4 2002-09-01  Owner-occupier  Purchase of newly erected dwellings      3457.0   

   loan_value_million  
0               819.8  
1              2017.9  
2              5139.6  
3             13991.4  
4               681.3  


In [56]:
df_final.head(10)

,date,segment,loan_purpose,loan_count,loan_value_million
0,2002-09-01,Owner-occupier,"Alterations, additions and repairs",NaN,819.8
1,2002-09-01,Owner-occupier,Construction of dwellings,9777.0,2017.9
2,2002-09-01,Owner-occupier,External refinancing,32454.0,5139.6
3,2002-09-01,Owner-occupier,Purchase of existing dwellings,77142.0,13991.4
4,2002-09-01,Owner-occupier,Purchase of newly erected dwellings,3457.0,681.3
5,2002-09-01,Owner-occupier,Total,89705.0,16698.5
6,2002-12-01,Owner-occupier,"Alterations, additions and repairs",NaN,1031.7
7,2002-12-01,Owner-occupier,Construction of dwellings,8713.0,1964.3
8,2002-12-01,Owner-occupier,External refinancing,33552.0,5442.1
9,2002-12-01,Owner-occupier,Purchase of existing dwellings,77467.0,14577.4


In [57]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                608 non-null    datetime64[ns]
 1   segment             608 non-null    object        
 2   loan_purpose        608 non-null    object        
 3   loan_count          540 non-null    float64       
 4   loan_value_million  608 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 23.9+ KB


# Table 4

In [64]:
import pandas as pd
from pathlib import Path

# 1. Setup paths
file_path = Path("../Raw Data/560104.xlsx")
output_file = "../Datasets/df_owner_occupier_state.csv"

# 2. Load the file
df_full = pd.read_excel(file_path, sheet_name="Data1")

# 3. Define the Series IDs for Table 4 (Seasonally Adjusted Only)
# Mapping: State -> {Count ID, Value ID}
mapping = {
    "Total Australia": {"count": "A130268886L", "value": "A130268885K"},
    "NSW":             {"count": "A130268884J", "value": "A130268883F"},
    "VIC":             {"count": "A130267888X", "value": "A130267887W"},
    "QLD":             {"count": "A130267886V", "value": "A130267885T"},
    "SA":              {"count": "A130268117V", "value": "A130268116T"},
    "WA":              {"count": "A130268624L", "value": "A130268623K"},
    "TAS":             {"count": "A130268253L", "value": "A130268252K"},
    "NT":              {"count": "A130268553R", "value": "A130268552L"},
    "ACT":             {"count": "A130268815C", "value": "A130268814A"}
}

# 4. Find the row index where Series IDs are located
# Searching for the National Total ID to find our header row
id_row_index = df_full[df_full.apply(lambda row: row.astype(str).str.contains('A130268886L').any(), axis=1)].index[0]

# 5. Prepare data starting from row after the IDs
df_data = df_full.iloc[id_row_index + 1:].copy()
df_data.columns = df_full.iloc[id_row_index]
df_data = df_data.rename(columns={df_data.columns[0]: 'date'})

# 6. Tidy up the data (Melting by State)
final_rows = []

for state_name, ids in mapping.items():
    try:
        # Extract date and the specific state columns
        subset = df_data[['date', ids['count'], ids['value']]].copy()
        subset.columns = ['date', 'loan_count', 'loan_value_million']
        
        # Add metadata
        subset['state'] = state_name
        subset['segment'] = "Owner-occupier"
        
        # --- REORDERING COLUMNS TO YOUR EXACT SPECIFICATION ---
        subset = subset[['date', 'segment', 'state', 'loan_count', 'loan_value_million']]
        
        final_rows.append(subset)
    except KeyError:
        print(f"Warning: Could not find IDs for {state_name}")
        continue

# 7. Final Clean and Sort
df_final = pd.concat(final_rows, ignore_index=True)
df_final['date'] = pd.to_datetime(df_final['date'])

# Convert metrics to numeric
df_final['loan_count'] = pd.to_numeric(df_final['loan_count'], errors='coerce')
df_final['loan_value_million'] = pd.to_numeric(df_final['loan_value_million'], errors='coerce')

# Drop rows that are completely empty
df_final = df_final.dropna(subset=['loan_count', 'loan_value_million'], how='all')

# Sort by date and then state for readability
df_final = df_final.sort_values(['date', 'state']).reset_index(drop=True)

# 8. Save to CSV
df_final.to_csv(output_file, index=False)

print(f"Extraction successful! Saved State data as {output_file}")
print(df_final.head(10))

Extraction successful! Saved State data as ../Datasets/df_owner_occupier_state.csv
        date         segment            state  loan_count  loan_value_million
0 2002-09-01  Owner-occupier              ACT        1727               327.4
1 2002-09-01  Owner-occupier              NSW       27906              6374.5
2 2002-09-01  Owner-occupier               NT         573                77.7
3 2002-09-01  Owner-occupier              QLD       19177              3216.3
4 2002-09-01  Owner-occupier               SA        8125              1061.2
5 2002-09-01  Owner-occupier              TAS        2282               224.0
6 2002-09-01  Owner-occupier  Total Australia       89705             16698.5
7 2002-09-01  Owner-occupier              VIC       19641              3759.9
8 2002-09-01  Owner-occupier               WA       10333              1623.0
9 2002-12-01  Owner-occupier              ACT        1863               358.9


In [59]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                837 non-null    datetime64[ns]
 1   segment             837 non-null    object        
 2   state               837 non-null    object        
 3   loan_count          837 non-null    int64         
 4   loan_value_million  837 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 32.8+ KB


# Table 13

In [65]:
import pandas as pd
from pathlib import Path

# 1. Setup paths
file_path = Path("../Raw Data/560113.xlsx")
output_file = "../Datasets/df_investor_purpose.csv"

# 2. Load the file
df_full = pd.read_excel(file_path, sheet_name="Data1")

# 3. Define the Series IDs for Table 13 (Seasonally Adjusted Only)
# Mapping: Purpose -> {Count ID, Value ID}
mapping = {
    "Total dwellings excluding refinancing": {"count": "A130268495C", "value": "A130268494A"},
    "Construction of dwellings":             {"count": "A130268555V", "value": "A130268554T"},
    "Purchase of newly erected dwellings":    {"count": "A130268261L", "value": "A130268260K"},
    "Purchase of existing dwellings":         {"count": "A130268048A", "value": "A130268047X"},
    "Purchase of residential land":           {"count": "A130268559C", "value": "A130268558A"},
    "Alterations, additions and repairs":     {"count": "A130268756R", "value": "A130268755L"},
    "External refinancing":                  {"count": "A130268347A", "value": "A130268346X"},
    "Internal refinancing":                  {"count": "A130268685T", "value": "A130268684R"}
}

# 4. Find the row index where Series IDs are located
# Searching for the Investor Total ID (A130268495C) to find the header row
id_row_index = df_full[df_full.apply(lambda row: row.astype(str).str.contains('A130268495C').any(), axis=1)].index[0]

# 5. Prepare data starting from row after the IDs
df_data = df_full.iloc[id_row_index + 1:].copy()
df_data.columns = df_full.iloc[id_row_index]
df_data = df_data.rename(columns={df_data.columns[0]: 'date'})

# 6. Tidy up the data (Melting by Purpose)
final_rows = []

for purpose, ids in mapping.items():
    try:
        # Extract date and the specific purpose columns
        subset = df_data[['date', ids['count'], ids['value']]].copy()
        subset.columns = ['date', 'loan_count', 'loan_value_million']
        
        # Add metadata
        subset['loan_purpose'] = purpose
        subset['segment'] = "Investor"
        
        # --- REORDERING COLUMNS TO YOUR EXACT SPECIFICATION ---
        # date | segment | loan_purpose | loan_count | loan_value_million
        subset = subset[['date', 'segment', 'loan_purpose', 'loan_count', 'loan_value_million']]
        
        final_rows.append(subset)
    except KeyError:
        print(f"Warning: Could not find IDs for {purpose}")
        continue

# 7. Final Clean and Sort
df_final = pd.concat(final_rows, ignore_index=True)
df_final['date'] = pd.to_datetime(df_final['date'])

# Convert metrics to numeric (converts empty strings to NaN)
df_final['loan_count'] = pd.to_numeric(df_final['loan_count'], errors='coerce')
df_final['loan_value_million'] = pd.to_numeric(df_final['loan_value_million'], errors='coerce')

# Drop rows where both count and value are missing 
# (This removes the "blank" years for purposes that only started in 2019)
df_final = df_final.dropna(subset=['loan_count', 'loan_value_million'], how='all')

# Sort by date and purpose
df_final = df_final.sort_values(['date', 'loan_purpose']).reset_index(drop=True)

# 8. Save to CSV
df_final.to_csv(output_file, index=False)

print(f"Extraction successful! Saved Investor Purpose data as {output_file}")
print(df_final.head(10))

Extraction successful! Saved Investor Purpose data as ../Datasets/df_investor_purpose.csv
        date   segment                           loan_purpose  loan_count  \
0 2002-09-01  Investor                   External refinancing         NaN   
1 2002-09-01  Investor  Total dwellings excluding refinancing         NaN   
2 2002-12-01  Investor                   External refinancing         NaN   
3 2002-12-01  Investor  Total dwellings excluding refinancing         NaN   
4 2003-03-01  Investor                   External refinancing         NaN   
5 2003-03-01  Investor  Total dwellings excluding refinancing         NaN   
6 2003-06-01  Investor                   External refinancing         NaN   
7 2003-06-01  Investor  Total dwellings excluding refinancing         NaN   
8 2003-09-01  Investor                   External refinancing         NaN   
9 2003-09-01  Investor  Total dwellings excluding refinancing         NaN   

   loan_value_million  
0              1127.8  
1             

In [61]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                336 non-null    datetime64[ns]
 1   segment             336 non-null    object        
 2   loan_purpose        336 non-null    object        
 3   loan_count          200 non-null    float64       
 4   loan_value_million  336 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 13.3+ KB


# RBA DATA

In [75]:
import pandas as pd

def process_rba_file(filename, target_ids):
    """
    Finds the 'Series ID' anchor in RBA CSVs and extracts the required columns.
    """
    # 1. Locate the 'Series ID' row index to skip the description header
    skiprows = 0
    with open(filename, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if 'Series ID' in line:
                skiprows = i
                break
    
    # 2. Read the data starting from the identified header
    df = pd.read_csv(filename, skiprows=skiprows)
    
    # 3. Standardize the Date column name and clean column strings
    df = df.rename(columns={df.columns[0]: 'date'})
    df.columns = [c.strip() for c in df.columns]
    
    # 4. Filter for only the target Series IDs and the date
    cols = ['date'] + [tid for tid in target_ids if tid in df.columns]
    df = df[cols]
    
    # 5. Convert date string to datetime objects (handling RBA's day-first format)
    df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
    
    # 6. Drop empty/metadata rows and convert values to numeric
    df = df.dropna(subset=['date'])
    for tid in target_ids:
        if tid in df.columns:
            df[tid] = pd.to_numeric(df[tid], errors='coerce')
            
    return df

# --- Execution ---

# Load F1 (Daily Money Market Data) and F6 (Monthly Lending Rates)
df_f1 = process_rba_file('../Raw Data/f1-data.csv', ['FIRMMCRTD'])
df_f1 = df_f1.rename(columns={'FIRMMCRTD': 'Cash_Rate'})

df_f6 = process_rba_file('../Raw Data/f6-data.csv', ['FLRHOOTA', 'FLRHIOTA'])
df_f6 = df_f6.rename(columns={'FLRHOOTA': 'Owner_Credit', 'FLRHIOTA': 'Investor_Credit'})

# --- Alignment & Merging ---

# Resample F1 from Daily to Monthly (takes the last recorded rate of each month)
# Note: In older pandas versions use 'M', in newer versions use 'ME'
df_f1_m = df_f1.set_index('date').resample('M').last().reset_index()

# Create a month-period key (e.g., '2019-07') to ensure exact alignment during merge
df_f1_m['month_key'] = df_f1_m['date'].dt.to_period('M')
df_f6['month_key'] = df_f6['date'].dt.to_period('M')

# Merge datasets on the month key
df_interest = pd.merge(df_f1_m, df_f6.drop(columns=['date']), on='month_key', how='inner')

# Calculate the Housing_Credit as the average of Owner and Investor rates
df_interest['Housing_Credit'] = df_interest[['Owner_Credit', 'Investor_Credit']].mean(axis=1)

# --- Final Cleanup ---

# Select final columns and sort by date
df_interest = df_interest[['date', 'Cash_Rate', 'Housing_Credit', 'Owner_Credit', 'Investor_Credit']]
df_interest = df_interest.sort_values('date')

# Save result
df_interest.to_csv('df_interest.csv', index=False)

print("Process Complete. Saved to 'df_interest.csv'.")
print(df_interest.head())

C:\Users\DeLL\AppData\Local\Temp\ipykernel_12544\2529110279.py:50: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_f1_m = df_f1.set_index('date').resample('M').last().reset_index()


Process Complete. Saved to 'df_interest.csv'.
        date  Cash_Rate  Housing_Credit  Owner_Credit  Investor_Credit
0 2019-07-31       1.00            4.05           3.8              4.3
1 2019-08-31       1.00            4.05           3.8              4.3
2 2019-09-30       1.00            4.05           3.8              4.3
3 2019-10-31       0.75            3.90           3.7              4.1
4 2019-11-30       0.75            3.85           3.7              4.0


In [76]:
df_interest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             77 non-null     datetime64[ns]
 1   Cash_Rate        77 non-null     float64       
 2   Housing_Credit   77 non-null     float64       
 3   Owner_Credit     77 non-null     float64       
 4   Investor_Credit  77 non-null     float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 3.1 KB
